In [ ]:
!pip install firebase-admin pyMuPDF transformers datasets google-auth

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import firebase_admin
from firebase_admin import credentials, storage
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import os
from google.cloud import storage
import pandas as pd
import google.oauth2.service_account

In [ ]:
import fitz  # PyMuPDF

def pdf2text(pdf_path):
    pdf_document = fitz.open(pdf_path)

    text = ""
    try:
      for page_num in range(5, 10):
          page = pdf_document.load_page(page_num)
          text += page.get_text()
    except:
      text = ""
    return text


In [ ]:
def download_pdfs_and_extract_text(google_auth_creds, temp_folder):

    client = storage.Client(credentials=google_auth_creds)
    bucket = client.bucket("apvo-file-storage-v1.appspot.com")
    blobs = bucket.list_blobs(projection="full", prefix="pdf/")
    data = []

    for blob in blobs:
        if blob.name.endswith('.pdf'):
            parts = os.path.normpath(blob.name).split(os.sep)
            folder_name = parts[1]
            label_name = folder_name[3:]
            pdf_file_path = f"{temp_folder}/{os.path.basename(blob.name)}"
            blob.download_to_filename(pdf_file_path)

            text = pdf2text(pdf_file_path)
            if text:
                data.append({'text': text, 'label': label_name})

    return data

In [ ]:
def save_dataset_to_csv(data, filepath, labels):

  sampled_data = []

  for label in labels:
    label_data = [item for item in data if item['label'] == label]

    train_data, _ = train_test_split(label_data, train_size=0.7, random_state=42)
    sampled_data.extend(train_data)

  return sampled_data

In [ ]:
json_cred_file = "<putanja>/<do>/<json>/<konfiguracije>/"
cred = credentials.Certificate(json_cred_file)
google_auth_creds = google.oauth2.service_account.Credentials.from_service_account_file(json_cred_file)
#firebase_admin.delete_app(firebase_admin.get_app())
firebase_admin.initialize_app(cred)

In [ ]:
temp_folder = '/content/tmp'

In [ ]:
if not os.path.exists(temp_folder):
    os.mkdir(temp_folder)

In [ ]:
# koristiti samo jednom za dohvacanje jer troši requeste na firebase koji se onda naplaćuju (više nije potrebno jer sam pokupio podatke)
data = download_pdfs_and_extract_text(google_auth_creds, temp_folder)

In [ ]:
raise Exception("Gotovo !!!!!!")

Exception: Gotovo !!!!!!

In [ ]:
labels = set(item['label'] for item in data)

In [ ]:
print(labels)

{'biomedicina-i-zdravstvo', 'tehnicke-znanosti', 'prirodne-znanosti', 'biotehnicke-znanosti', 'humanisticke-znanosti', 'drustvene-znanosti'}


In [ ]:
from sklearn.model_selection import train_test_split
data_sample = save_dataset_to_csv(data, "/content/dataset.csv", labels)

In [ ]:
df = pd.DataFrame(data_sample)

df.to_csv('/content/sampled_data.csv', index=False, escapechar='\\')

In [11]:
# samo ako se ucitava iz datoteke, inace ne treba jer se prije spremi u data_sample
data_sample = pd.read_csv("/content/sampled_data.csv", escapechar="\\")

In [33]:
# radi samo ako se podaci ucitaju iz datoteke
for d in data_sample.itertuples():
  print(d.text)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [38]:
def prepare_dataset(data):
    #otkomenitirati prve dvije linija i zakomentirati druge dvije samo ako se dohvacaju podaci iz datoteke
    #texts = [item.text for item in data.itertuples()]
    #labels = [item.label for item in data.itertuples()]
    texts = [item["text"] for item in data]
    labels = [item["label"] for item in data]

    unique_labels = list(set(labels))
    label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
    id_to_label = {idx: label for label, idx in label_to_id.items()}

    encoded_labels = [label_to_id[label] for label in labels]

    dataset = Dataset.from_dict({
        'text': texts,
        'label': encoded_labels
    })

    return dataset, label_to_id, id_to_label

In [39]:
dataset, label_to_id, id_to_label = prepare_dataset(data_sample)

In [40]:
tokenizer = AutoTokenizer.from_pretrained("classla/bcms-bertic")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [41]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/495 [00:00<?, ? examples/s]

In [42]:
model = AutoModelForSequenceClassification.from_pretrained("classla/bcms-bertic", num_labels=len(label_to_id))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # For simplicity, using the same dataset
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.724133
2,No log,1.671440
3,No log,1.641531


TrainOutput(global_step=93, training_loss=1.7144942745085685, metrics={'train_runtime': 186.8226, 'train_samples_per_second': 7.949, 'train_steps_per_second': 0.498, 'total_flos': 390733949675520.0, 'train_loss': 1.7144942745085685, 'epoch': 3.0})

In [46]:
model_dir = "./saved_model"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [47]:
model.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [50]:
new_texts = ["""
Fenomen obrnute diskriminacije u Europskoj uniji u fokusu je akademskih istraživanja i
rasprava zbog nejednakosti koju uzrokuje među građanima Europske unije, te nastavlja
izazivati kontroverze. Do obrnute diskriminacije dolazi kada država članica svoje državljane
koji ne mogu ukazati na poveznicu s pravom Unije stavlja u nepovoljniji položaj od onih koji
uživaju zaštitu prava Unije. Takva razlika u postupanju dovodi do dvostrukih standarda u zaštiti
temeljnih prava namijenjenih građanima Unije i potkopava europsko načelo jednakosti. U isto
vrijeme, razvoj građanstva Europske unije sugerira da ovaj pravni status ima potencijal za borbu
protiv nejednakosti. Naime, Sud Europske unije u svojoj je praksi razvio građanstvo Europske
unije kao temeljni status državljana država članica, izazivajući mnoga zanimljiva i potencijalno
revolucionarna tumačenja statusa građanstva u smislu širenja nadležnosti Unije u zaštiti
temeljnih prava. U tom kontekstu čini se relevantnim istražiti opseg i transformacijski
potencijal građanstva Unije koji odbacuje supstitucijsku tržišnu paradigmu i prihvaća jednakost
kao svoju osnovnu svrhu. U radu se raspravlja o mogućnosti korištenja odredbi Osnivačkih
ugovora koje se odnosne na građanstvo kao pravne osnove za dokidanje obrnute diskriminacije
i uspostavljanje nadležnosti Unije u isključivo unutarnjim situacijama. Kako bi se objasnili
mogući učinci ovakvog razvoja, povlači se paralela s američkom ustavnom doktrinom o
inkorporaciji. U radu se stoga brani teza da bi posljedice primjene standarda temeljnih prava
Europske unije na temelju statusa građanstva Unije u isključivo unutarnjim situacijama bile
slične učincima doktrine o inkorporaciji u Sjedinjenim Američkim Državama. Takav mogući
razvoj u pravu Europske unije, iako normativno poželjan, bio bi suprotan trenutnom ustavnom
uređenju Unije. Odgovarajući forumi za borbu protiv obrnute diskriminacije, na temelju načela
jednakosti koje je zajedničko državama članicama, u ovome trenutku su samo nacionalni sudovi
ili Europski sud za ljudska prava, djelujući van pravnih okvira Europske unije. Ipak, reforma
sustava zaštite temeljnih prava u Europskoj uniji je dobrodošla jer je to jedini način da se
učinkovito stane na kraj ovoj vrsti nejednakosti. S obzirom da pojava obrnute diskriminacije
narušava temeljne materijalne vrijednosti i pravna načela Europske unije, više se kao takva ne
bi smjela tolerirati."""]
tokenizer = AutoTokenizer.from_pretrained("/content/saved_model")
tokenized_texts = tokenizer(new_texts, truncation=True, padding=True, return_tensors="pt")

In [52]:
for t in tokenized_texts:
  print(t)

input_ids
token_type_ids
attention_mask


In [51]:
predictions = trainer.predict(tokenized_texts)
predicted_labels = predictions.predictions.argmax(-1)

IndexError: list index out of range